In [1]:
import os

In [2]:
%pwd

'c:\\CDAC_PROJECT\\Final_project_retinopathy\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'c:\\CDAC_PROJECT\\Final_project_retinopathy'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training

    
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "training")
        create_directories([
            Path(training.root_dir)
        ])
        # testing_data = os.path.join(self.config.data_ingestion.unzip_dir, "testing")
        # create_directories([
        #     Path(testing_data.root_dir)])
        

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            # testing_data=Path(testing_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,

        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
        )

        train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    def test_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,

        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
        )

        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        testdir=r"C:/CDAC_PROJECT/Final_project_retinopathy/artifacts/data_ingestion/testing"

        self.test_generator = test_datagenerator.flow_from_directory(
            directory=testdir,
            subset="testing",
            shuffle=True,
            **dataflow_kwargs
        )    
        

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):


        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_data=self.test_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.test_valid_generator
    training.train()
    
except Exception as e:
    raise e


[2025-02-02 21:28:14,285: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-02 21:28:14,295: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-02 21:28:14,299: INFO: common: created directory at: artifacts]
[2025-02-02 21:28:14,303: INFO: common: created directory at: artifacts\training]
Found 1120 images belonging to 5 classes.


AttributeError: 'Training' object has no attribute 'test_generator'